In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys

# --- Configuration (Adjusted for Debugging) ---
# IMPORTANT: Update these paths to your actual Firebase Admin SDK JSON files
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0          
# TEMPORARY FIX: Set deviation high to force a match and debug the source of 8.75km
MAX_DESTINATION_DEVIATION_KM = 100.0 
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0    

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders'             
PASSENGER_PREFERENCES_COLLECTION = 'passengers' 
RIDE_REQUESTS_COLLECTION = 'public_ride_requests' 

# CRITICAL DRIVER STATUSES
RIDER_POOLING_STATUS = 'on_route_to_original_destination'
RIDER_AVAILABLE_STATUS = 'available' 

PROPOSAL_COLLECTION_NAME = 'driver_proposals' 

# Ride Request States
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance (Haversine) between two coordinates in kilometers."""
    R = 6371  # Radius of Earth in kilometers
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """Calculates a dynamic pickup point (midway) if the passenger is very close to the driver."""
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    """Checks if the passenger's destination is near the driver's current passenger's destination."""
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    # *** ADDED LOGGING FOR DEBUGGING 8.75km ISSUE ***
    print(f"  [DEBUG] Driver Dest: ({driver_dest.latitude:.4f}, {driver_dest.longitude:.4f})")
    print(f"  [DEBUG] Passenger Dest: ({passenger_dest.latitude:.4f}, {passenger_dest.longitude:.4f})")
    print(f"  [DEBUG] Calculated Dist: {dist:.2f} km")
    # ************************************************
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).") 
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False
        
def fetch_passenger_details(pdb, passenger_id):
    """Fetches detailed passenger preferences."""
    try:
        pref_doc = pdb.collection(PASSENGER_PREFERENCES_COLLECTION).document(passenger_id).get()
        if pref_doc.exists:
            data = pref_doc.to_dict()
            return {
                'preferred_vehicle_type': data.get('preferred_vehicle_type'),
                'gender': data.get('gender'),
                'min_driver_age': data.get('min_driver_age', 0),
                'max_driver_age': data.get('max_driver_age', 100),
            }
        else:
            return {}
    except Exception as e:
        print(f"  [ERROR] Failed to fetch passenger preferences: {e}", file=sys.stderr)
        return {}


# --- Negotiation & Finalization Handlers (Unchanged) ---

def finalize_match(pdb, ddb, proposal_data):
    """Updates all documents when a driver ACCEPTS the proposed ride."""
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid'] # Passenger ID
    
    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('public_ride_requests') to ACCEPTED
        ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED,  
            'driverId': driver_id,
            'matchedAt': firestore.SERVER_TIMESTAMP,
            'finalPickupLocation': final_pickup_loc,  
            'driverName': proposal_data.get('driverName', 'Driver')  
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver status must change to ON_ROUTE_TO_PICKUP
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP,
            'currentPassengerId': rider_uid,  
            'pooledRideId': ride_id,        
            'nextTargetLocation': final_pickup_loc  
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """Reverts passenger status to PENDING when a driver REJECTS the proposed ride."""
    ride_id = proposal_data['rideId']

    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('public_ride_requests') back to PENDING/UNMATCHED
        ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Finds a suitable driver by checking for both: 
    1. Available (direct match) 
    2. En route (pooling match)
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Retrieving data using the exact keys from your logs
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    passenger_id = request_data.get('passengerId') 
    
    if not (passenger_pickup_loc and passenger_dest_loc and passenger_id):
        print(f"[ERROR] Ride {ride_id} is missing location or passengerId. Skipping.")
        return

    # 1A: Fetch Passenger Preferences (for filtering)
    passenger_details = fetch_passenger_details(pdb, passenger_id)
    pref_vehicle = passenger_details.get('preferred_vehicle_type', request_data.get('vehiclePreference', 'Any')).lower()
    pref_gender = passenger_details.get('gender')
    min_age = passenger_details.get('min_driver_age', 0)
    max_age = passenger_details.get('max_driver_age', 100)
    
    # Extract display details
    passenger_name_for_proposal = request_data.get('passengerName', 'Rider')
    passenger_rating_for_proposal = request_data.get('passengerRating', 'New')
    fare_for_proposal = request_data.get('fareAmount', 0) 
    pickup_address_for_proposal = request_data.get('pickupAddress', 'Unknown Pickup')
    destination_address_for_proposal = request_data.get('destinationAddress', 'Unknown Destination')
    payment_method_for_proposal = request_data.get('paymentMethod', 'Cash')

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    # 1B: Query for both available (direct) and en-route (pooling) drivers
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_AVAILABLE_STATUS)
    pooling_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_POOLING_STATUS)
    
    driver_list = list(available_drivers_query.stream()) + list(pooling_drivers_query.stream())
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf') 

    # --- Filtering and Selection Loop ---
    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        driver_status = driver_data.get('status')
        
        driver_name = driver_data.get('name', 'Driver')
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        
        # Mock/Assume driver fields for filtering demonstration
        driver_age = driver_data.get('age', 35) 
        driver_gender = driver_data.get('gender', 'Male') 
        
        # Determine the locations based on the driver's status
        if driver_status == RIDER_POOLING_STATUS:
            # Pooling drivers need a route start (current location) and an end (current passenger's destination)
            driver_current_loc = driver_data.get('currentRouteStart') or driver_data.get('currentLocation')
            driver_dest_loc = driver_data.get('currentRouteEnd')      
        else: # RIDER_AVAILABLE_STATUS (Direct Match)
            # Available drivers' pickup location is their current location; destination is irrelevant for overlap
            driver_current_loc = driver_data.get('currentLocation') 
            driver_dest_loc = None # Not needed for destination check on available drivers

        if not driver_current_loc:
            continue

        total_drivers_checked += 1
        
        # 2A-2C: Preference Filters
        if pref_vehicle != 'any' and pref_vehicle != driver_vehicle_type: continue
        if pref_gender and pref_gender != 'Any' and pref_gender != driver_gender: continue
        if not (min_age <= driver_age <= max_age): continue
            
        # --- GEOSPATIAL FILTERS ---
        
        # 3. Check Destination Overlap (Only for Pooling Drivers)
        if driver_status == RIDER_POOLING_STATUS:
            if not driver_dest_loc or not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
                continue

        # 4. Check Initial Proximity (Driver to Passenger Pickup)
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {driver_id} ({driver_status}): Pickup too far ({pickup_dist:.2f} km).")
            continue

        # 5. Greedy Selection
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_name': driver_name,
                'is_pooling': (driver_status == RIDER_POOLING_STATUS) # Flag the type of match
            }
            print(f"  Found best potential match {driver_id} ({driver_name}) at {pickup_dist:.2f} km. Type: {'POOLING' if best_match['is_pooling'] else 'DIRECT'}")

    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # 6. Calculate Dynamic Pickup Point
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        distance_to_pickup_km = min_pickup_dist
        eta_to_pickup_minutes = int(min_pickup_dist * 2) 
        
        try:
            # 1. Update Passenger DB status
            ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matchedDriverId': driver_id,  
                'proposedAt': firestore.SERVER_TIMESTAMP
            })

            # 2. Create PROPOSAL document in Driver DB 
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            
            proposal_data_to_set = {
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'riderUid': passenger_id, 
                'status': 'pending_acceptance', 
                'isPoolingRide': best_match['is_pooling'], 
                
                # Geospatial Details
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                
                # --- PASSENGER and RIDE DETAILS FOR DRIVER UI (using corrected keys) ---
                'passenger_name': passenger_name_for_proposal,
                'passenger_rating': passenger_rating_for_proposal,
                'pickup_address': pickup_address_for_proposal,
                'destination_address': destination_address_for_proposal,
                'fare_amount': fare_for_proposal, 
                'payment_method': payment_method_for_proposal,
                'distance_to_pickup': f"{distance_to_pickup_km:.2f} km",
                'eta_to_pickup': f"{eta_to_pickup_minutes} min",         
                'request_time': request_data.get('requestTimestamp') or firestore.SERVER_TIMESTAMP,
                'passenger_count': request_data.get('passengerCount', 1), 
                'luggage_count': request_data.get('luggageCount', 0), 
                # -----------------------------------------------
                
                'vehicleType': driver_data.get('vehicleType', 'Any'),
                'proposalTimestamp': firestore.SERVER_TIMESTAMP
            }

            proposal_ref.set(proposal_data_to_set)
            print(f"  [PROPOSAL] Sent to Driver {driver_id} for Ride {ride_id}. (Pooling: {best_match['is_pooling']})")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}", file=sys.stderr)
            
    else:
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id}.")


# --- Snapshot Listeners ---

# Global DB clients initialized in __main__
db_passenger = None
db_driver = None

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """Listener 1: Detects new requests with status 'pending' (from the Passenger App)."""
    global db_passenger, db_driver # Access global DB clients
    
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New or reverted PENDING request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """Listener 2: Detects driver response (acceptance/rejection) from the Driver App."""
    global db_passenger, db_driver # Access global DB clients
    
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            
            if data and data.get('status') == 'accepted':
                print(f"[EVENT] Driver accepted proposal: {change.document.id}")
                finalize_match(db_passenger, db_driver, data)
            elif data and data.get('status') == 'rejected':
                print(f"[EVENT] Driver rejected proposal: {change.document.id}")
                handle_rejection(db_passenger, db_driver, data)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs (assign to global variables)
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests (Passenger DB: public_ride_requests)
        query_pending = db_passenger.collection(RIDE_REQUESTS_COLLECTION).where('status', '==', STATUS_PENDING)
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses (Driver DB: driver_proposals)
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started.")
        print(f"Driver Statuses checked: '{RIDER_AVAILABLE_STATUS}' (Direct) and '{RIDER_POOLING_STATUS}' (Pooling)")
        print(f"*** DEBUG MODE: MAX_DESTINATION_DEVIATION_KM set to 100.0 to force match. ***")
        print("Press Ctrl+C to stop the server.")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)

Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 100.0 km)...
-------------------------------------------------------
Matcher Server: Two listeners started.
Driver Statuses checked: 'available' (Direct) and 'on_route_to_original_destination' (Pooling)
*** DEBUG MODE: MAX_DESTINATION_DEVIATION_KM set to 100.0 to force match. ***
Press Ctrl+C to stop the server.
-------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[EVENT] New or reverted PENDING request detected: LbD39VywMeAMqTpukmCD

[MATCHING] Searching for driver for Ride ID: LbD39VywMeAMqTpukmCD

[NO MATCH] No suitable driver found for Ride LbD39VywMeAMqTpukmCD.
[EVENT] New or reverted PENDING request detected: wZ8ZckGRSRZFiGMm9UDy

[MATCHING] Searching for driver for Ride ID: wZ8ZckGRSRZFiGMm9UDy

[NO MATCH] No suitable driver found for Ride wZ8ZckGRSRZFiGMm9UDy.
[EVENT] New or reverted PENDING request detected: kszDjNCnOmYows3jEJ6C

[MATCHING] Searching for driver for Ride ID: kszDjNCnOmYows3jEJ6C

[NO MATCH] No suitable driver found for Ride kszDjNCnOmYows3jEJ6C.


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys

# --- Configuration (Adjusted for Realistic Deviation) ---
# IMPORTANT: Update these paths to your actual Firebase Admin SDK JSON files
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0        
# Reverting to a sensible maximum deviation for pooling (e.g., 2.0 km)
MAX_DESTINATION_DEVIATION_KM = 2.0 
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0    

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders'          
PASSENGER_PREFERENCES_COLLECTION = 'passengers'
RIDE_REQUESTS_COLLECTION = 'public_ride_requests' 

# CRITICAL DRIVER STATUSES
RIDER_POOLING_STATUS = 'on_route_to_original_destination'
RIDER_AVAILABLE_STATUS = 'available' 

PROPOSAL_COLLECTION_NAME = 'driver_proposals' 

# Ride Request States
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance (Haversine) between two coordinates in kilometers."""
    R = 6371  # Radius of Earth in kilometers
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """Calculates a dynamic pickup point (midway) if the passenger is very close to the driver."""
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    """Checks if the passenger's destination is near the driver's current passenger's destination."""
    
    if not isinstance(driver_dest, GeoPoint) or not isinstance(passenger_dest, GeoPoint):
        print("  [ERROR] Destination overlap check failed: Invalid GeoPoint types.")
        return False

    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).") 
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False
        
def fetch_passenger_details(pdb, passenger_id):
    """Fetches detailed passenger preferences."""
    try:
        pref_doc = pdb.collection(PASSENGER_PREFERENCES_COLLECTION).document(passenger_id).get()
        if pref_doc.exists:
            data = pref_doc.to_dict()
            return {
                'preferred_vehicle_type': data.get('preferred_vehicle_type', 'Any'),
                'gender': data.get('gender', 'Any'),
                'min_driver_age': data.get('min_driver_age', 0),
                'max_driver_age': data.get('max_driver_age', 100),
            }
        else:
            return {'preferred_vehicle_type': 'Any', 'gender': 'Any', 'min_driver_age': 0, 'max_driver_age': 100}
    except Exception as e:
        print(f"  [ERROR] Failed to fetch passenger preferences: {e}", file=sys.stderr)
        return {'preferred_vehicle_type': 'Any', 'gender': 'Any', 'min_driver_age': 0, 'max_driver_age': 100}

def fetch_rider_profile(ddb, rider_id):
    """
    Fetches full rider (passenger) profile from the Driver DB's 'riders' collection 
    to retrieve name and phone number which may be null in the ride request.
    """
    try:
        # We use the Driver DB (ddb) which hosts the 'riders' collection
        rider_doc = ddb.collection(RIDER_COLLECTION_NAME).document(rider_id).get()
        if rider_doc.exists:
            data = rider_doc.to_dict()
            return data
        else:
            return {}
    except Exception as e:
        print(f"  [ERROR] Failed to fetch rider profile: {e}", file=sys.stderr)
        return {}


# --- Negotiation & Finalization Handlers (Unchanged) ---

def finalize_match(pdb, ddb, proposal_data):
    """Updates all documents when a driver ACCEPTS the proposed ride."""
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid'] # Passenger ID
    
    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('public_ride_requests') to ACCEPTED
        ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED,  
            'driverId': driver_id,
            'matchedAt': firestore.SERVER_TIMESTAMP,
            'finalPickupLocation': final_pickup_loc,  
            'driverName': proposal_data.get('driverName', 'Driver')  
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver status must change to ON_ROUTE_TO_PICKUP
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP,
            'currentPassengerId': rider_uid,  
            'pooledRideId': ride_id,        
            'nextTargetLocation': final_pickup_loc  
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """Reverts passenger status to PENDING when a driver REJECTS the proposed ride."""
    ride_id = proposal_data['rideId']

    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('public_ride_requests') back to PENDING/UNMATCHED
        ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)


# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """
    Finds a suitable driver by checking for both: 
    1. Available (direct match) 
    2. En route (pooling match)
    """
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # --- Input Validation ---
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    passenger_id = request_data.get('passengerId') 
    
    if not (passenger_pickup_loc and passenger_dest_loc and passenger_id):
        print(f"[ERROR] Ride {ride_id} is missing location or passengerId. Skipping.")
        return
    
    if not isinstance(passenger_pickup_loc, GeoPoint) or not isinstance(passenger_dest_loc, GeoPoint):
        print(f"[ERROR] Ride {ride_id}: Pickup/Destination location is not a GeoPoint type. Skipping.")
        return

    # 1A: Fetch Passenger Preferences (for filtering)
    passenger_details = fetch_passenger_details(pdb, passenger_id)
    pref_vehicle = passenger_details.get('preferred_vehicle_type', request_data.get('vehiclePreference', 'Any')).lower()
    pref_gender = passenger_details.get('gender')
    min_age = passenger_details.get('min_driver_age', 0)
    max_age = passenger_details.get('max_driver_age', 100)
    
    
    # --- 1B: EXTRACT CRITICAL RIDE DATA FROM REQUEST (public_ride_requests) ---
    
    # Fields explicitly requested by the user to be sourced from public_ride_requests:
    request_passenger_phone = request_data.get('passengerPhone')
    request_otp = request_data.get('otp')
    
    # Other ride-specific data
    route_to_dest_encoded = request_data.get('route_to_destination_encoded')
    route_to_pickup_encoded = request_data.get('route_to_pickup_encoded')
    passenger_rating_for_proposal = request_data.get('passengerRating', 'New')
    fare_for_proposal = request_data.get('fareAmount', 0) 
    pickup_address_for_proposal = request_data.get('pickupAddress', 'Unknown Pickup')
    destination_address_for_proposal = request_data.get('destinationAddress', 'Unknown Destination')
    payment_method_for_proposal = request_data.get('paymentMethod', 'Cash')
    
    # --- 1C: CONSOLIDATE NAME/PHONE (using profile as fallback if request is null) ---
    rider_profile_data = fetch_rider_profile(ddb, passenger_id)
    
    # Consolidate Rider Name: Try profile, then request, then default
    passenger_name_for_proposal = (
        rider_profile_data.get('riderName') or 
        rider_profile_data.get('name') or 
        request_data.get('passengerName') or 
        'Unknown Rider'
    )
    
    # Consolidate Rider Phone: Prioritize phone from the request (for call), then profile, then default
    rider_phone_from_profile = rider_profile_data.get('phone') or rider_profile_data.get('riderPhone')
    final_rider_phone = request_passenger_phone or rider_phone_from_profile or 'N/A'


    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    # 2: Fetching drivers by status.
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_AVAILABLE_STATUS)
    pooling_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_POOLING_STATUS)
    
    # Concatenate the lists of documents found
    driver_list = list(available_drivers_query.stream()) + list(pooling_drivers_query.stream())
    
    print(f"  [DRIVER FETCH] Found {len(driver_list)} potential drivers (available/pooling).")

    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf') 

    # --- Filtering and Selection Loop ---
    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        driver_status = driver_data.get('status')
        
        driver_name = driver_data.get('name', driver_data.get('riderName', 'Driver')) 
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        driver_age = driver_data.get('age', 35) 
        driver_gender = driver_data.get('gender', 'Male') 
        
        driver_current_loc = driver_data.get('currentLocation')
        driver_dest_loc = driver_data.get('currentRouteEnd') 

        # ... (Geospatial and Preference Filters) ...
        
        if not driver_current_loc or not isinstance(driver_current_loc, GeoPoint):
            continue
        
        total_drivers_checked += 1
        
        if pref_vehicle != 'any' and pref_vehicle != driver_vehicle_type:
            continue
        
        if pref_gender != 'Any' and pref_gender != driver_gender:
            continue
        
        if not (min_age <= driver_age <= max_age):
            continue
            
        # Check Destination Overlap (Only for Pooling Drivers)
        if driver_status == RIDER_POOLING_STATUS:
            if not driver_dest_loc or not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
                continue
        elif driver_status != RIDER_AVAILABLE_STATUS:
            continue


        # Check Initial Proximity (Driver's current location to Passenger Pickup)
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            continue

        # Greedy Selection (Best Match is the closest one)
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc, 
                'driver_name': driver_name,
                'is_pooling': (driver_status == RIDER_POOLING_STATUS) 
            }

    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        
        # Calculate Dynamic Pickup Point
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        distance_to_pickup_km = min_pickup_dist
        eta_to_pickup_minutes = max(1, int(min_pickup_dist * 2)) 
        
        try:
            # 1. Update Passenger DB status
            ride_ref = pdb.collection(RIDE_REQUESTS_COLLECTION).document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matchedDriverId': driver_id,  
                'proposedAt': firestore.SERVER_TIMESTAMP
            })

            # 2. Create PROPOSAL document in Driver DB 
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            
            proposal_data_to_set = {
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'status': 'pending_acceptance', 
                'isPoolingRide': best_match['is_pooling'], 
                
                # Geospatial Details
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                
                # --- PASSENGER/RIDER DETAILS FOR DRIVER UI ---
                'riderUid': passenger_id,
                'riderName': passenger_name_for_proposal, 
                'riderLocation': passenger_pickup_loc, 
                'riderPhone': final_rider_phone, # Consolidated phone number (best available)
                
                # *** NEW FIELDS AS REQUESTED ***
                'passenger_phone': request_passenger_phone, # Phone number from the ride request (most recent/accurate for that trip)
                'otp': request_otp,                          # Verification OTP
                # ******************************
                
                'route_to_destination_encoded': route_to_dest_encoded,
                'route_to_pickup_encoded': route_to_pickup_encoded,
                
                # Other ride details
                'passenger_rating': passenger_rating_for_proposal,
                'pickup_address': pickup_address_for_proposal,
                'destination_address': destination_address_for_proposal,
                'fare_amount': fare_for_proposal, 
                'payment_method': payment_method_for_proposal,
                'distance_to_pickup': f"{distance_to_pickup_km:.2f} km",
                'eta_to_pickup': f"{eta_to_pickup_minutes} min",         
                'request_time': request_data.get('requestTimestamp') or firestore.SERVER_TIMESTAMP,
                'passenger_count': request_data.get('passengerCount', 1), 
                'luggage_count': request_data.get('luggageCount', 0), 
                
                'vehicleType': best_match['driver_data'].get('vehicleType', 'Any'),
                'proposalTimestamp': firestore.SERVER_TIMESTAMP
            }

            proposal_ref.set(proposal_data_to_set)
            print(f"  [PROPOSAL] Sent to Driver {driver_id} for Ride {ride_id}. (OTP: {request_otp})")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}", file=sys.stderr)
            
    else:
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id}.")


# --- Snapshot Listeners ---

# Global DB clients initialized in __main__
db_passenger = None
db_driver = None

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """Listener 1: Detects new requests with status 'pending' (from the Passenger App)."""
    global db_passenger, db_driver 
    
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """Listener 2: Detects driver response (acceptance/rejection) from the Driver App."""
    global db_passenger, db_driver 
    
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            
            if data and data.get('status') == 'accepted':
                finalize_match(db_passenger, db_driver, data)
            elif data and data.get('status') == 'rejected':
                handle_rejection(db_passenger, db_driver, data)


# --- Main Execution ---

if __name__ == '__main__':
    
    # 1. Initialize DBs (assign to global variables)
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)
    
    # --- MANUAL SANITY CHECK ---
    print("-" * 55)
    print("Sanity Check: Verifying Distance Calculation:")
    p_lat, p_lon = 12.9972407, 77.5263324
    d_lat, d_lon = 12.9971842, 77.5263909
    test_dist = calculate_distance(d_lat, d_lon, p_lat, p_lon)
    print(f"  -> Calculated Distance: {test_dist:.4f} km")
    print(f"  -> RESULT: {'SUCCESS' if test_dist < MAX_MATCH_DISTANCE_KM else 'FAILED'}")
    print("-" * 55)
    # ------------------------------------------------------------------

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests (Passenger DB: public_ride_requests)
        query_pending = db_passenger.collection(RIDE_REQUESTS_COLLECTION).where('status', '==', STATUS_PENDING)
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses (Driver DB: driver_proposals)
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started.")
        print(f"*** Added 'passenger_phone' and 'otp' to driver proposals. ***")
        print("Press Ctrl+C to stop the server.")
        print("-" * 55)

        # Keep the main thread alive to listen for updates
        while True:
            time.sleep(1) 

    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys
import random

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0
MAX_DESTINATION_DEVIATION_KM = 5.0
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0

# Firestore Collection Names
RIDER_COLLECTION_NAME = 'riders'
PROPOSAL_COLLECTION_NAME = 'driver_proposals'

# Ride Statuses
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_PENDING_ACCEPTANCE = 'pending_acceptance'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# Mock address data
MOCK_ADDRESSES = {
    '12.9716,77.5946': 'MG Road, Bangalore',
    '12.9352,77.6245': 'Koramangala, Bangalore'
}


# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    if not os.path.exists(cred_path):
        print(f"Error: Missing Firebase credentials file: {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        try:
            app = get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        db_client = firestore.client(app)
        print(f"[INIT] Firebase app '{name}' initialized successfully.")
        return db_client
    except Exception as e:
        print(f"[ERROR] Firebase init failed for {name}: {e}", file=sys.stderr)
        return None


def calculate_distance(lat1, lon1, lat2, lon2):
    """Haversine distance in kilometers"""
    R = 6371
    dlon = radians(lon2 - lon1)
    dlat = radians(lat2 - lat1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))


def calculate_dynamic_pickup(driver_loc, passenger_loc):
    dist = calculate_distance(driver_loc.latitude, driver_loc.longitude,
                              passenger_loc.latitude, passenger_loc.longitude)
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [LOGIC] Dynamic pickup midpoint ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    print(f"  [LOGIC] Using passenger pickup ({dist:.2f} km apart).")
    return passenger_loc


def check_destination_overlap(driver_dest, passenger_dest):
    dist = calculate_distance(driver_dest.latitude, driver_dest.longitude,
                              passenger_dest.latitude, passenger_dest.longitude)
    print(f"  [CHECK] Destination deviation: {dist:.2f} km")
    return dist <= MAX_DESTINATION_DEVIATION_KM


def generate_mock_fare(distance_km):
    base_fare, rate_per_km = 50, 20
    return (
        round(base_fare + distance_km * rate_per_km + random.uniform(5, 15), 2),
        int(distance_km * 3) + random.randint(5, 10),
        str(random.randint(1000, 9999))
    )


# --- Proposal Creation ---

def create_driver_proposal(ddb, ride_id, passenger_data, driver_data, metrics):
    """Creates a new driver_proposals document per the new schema."""
    proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)

    data = {
        # Basic Request Info
        'request_id': ride_id,
        'status': STATUS_PENDING_ACCEPTANCE,
        'createdAt': firestore.SERVER_TIMESTAMP,

        # Passenger Info
        'riderUid': passenger_data['uid'],
        'riderName': passenger_data['name'],
        'riderPhone': passenger_data['phone'],

        # Location Data
        'pickupLocation': passenger_data['pickup'],
        'destinationLocation': passenger_data['destination'],
        'pickup_address': passenger_data['pickup_address'],
        'destination_address': passenger_data['destination_address'],

        # Rider Assignment
        'riderUid': driver_data['uid'],      # Must match current driver UID
        'driverId': driver_data['uid'],
        'driverName': driver_data['name'],

        # Tracking
        'riderLocation': driver_data['loc'],
        'lastLocationUpdate': firestore.SERVER_TIMESTAMP,

        # Ride Details
        'otp': metrics['otp'],
        'estimatedFare': metrics['fare'],
        'estimatedDuration': metrics['duration'],
        'vehicleType': driver_data.get('vehicleType', 'bike'),

        # Timestamp placeholders
        'acceptedTimestamp': None,
        'arrivalTimestamp': None,
        'pickupTimestamp': None,
        'completionTimestamp': None,
        'cancellationTimestamp': None,
    }

    proposal_ref.set(data)
    print(f"  [DB] Proposal created for driver {driver_data['uid']} (Ride ID: {ride_id}).")


# --- Match Driver Logic ---

def match_driver(pdb, ddb, ride_snapshot):
    """Matches a passenger ride to the best driver and sends proposal."""
    ride_id = ride_snapshot.id
    req = ride_snapshot.to_dict()

    pickup = req.get('pickupLocation')
    destination = req.get('destinationLocation')

    if not pickup or not destination:
        print(f"[SKIP] Ride {ride_id} missing locations.")
        return

    passenger_data = {
        'uid': req.get('passengerId', 'unknown_passenger'),
        'name': req.get('riderName', 'Passenger'),
        'phone': req.get('riderPhone', 'N/A'),
        'pickup': pickup,
        'destination': destination,
        'pickup_address': req.get('pickup_address', MOCK_ADDRESSES.get(f"{pickup.latitude},{pickup.longitude}", 'Unknown Pickup')),
        'destination_address': req.get('destination_address', MOCK_ADDRESSES.get(f"{destination.latitude},{destination.longitude}", 'Unknown Destination')),
    }

    dist_km = calculate_distance(pickup.latitude, pickup.longitude, destination.latitude, destination.longitude)
    fare, duration, otp = generate_mock_fare(dist_km)
    metrics = {'fare': fare, 'duration': duration, 'otp': otp}

    print(f"\n[MATCH] Looking for drivers for Ride {ride_id}")

    drivers = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', 'on_route_to_original_destination').stream()
    best_driver, min_distance = None, float('inf')

    for doc in drivers:
        driver = doc.to_dict()
        driver_id = doc.id
        if not driver.get('currentRouteStart') or not driver.get('currentRouteEnd'):
            continue

        if not check_destination_overlap(driver['currentRouteEnd'], destination):
            continue

        pickup_dist = calculate_distance(driver['currentRouteStart'].latitude,
                                         driver['currentRouteStart'].longitude,
                                         pickup.latitude, pickup.longitude)
        if pickup_dist < min_distance and pickup_dist <= MAX_MATCH_DISTANCE_KM:
            min_distance = pickup_dist
            best_driver = {'uid': driver_id, 'name': driver.get('name', 'Driver'),
                           'loc': driver['currentRouteStart'],
                           'vehicleType': driver.get('vehicleType', 'bike')}
    if best_driver:
        print(f"  [FOUND] Closest driver: {best_driver['uid']} ({min_distance:.2f} km away)")
        create_driver_proposal(ddb, ride_id, passenger_data, best_driver, metrics)
        pdb.collection('ride_requests').document(ride_id).update({'status': STATUS_PROPOSED})
    else:
        print(f"  [NO MATCH] No available driver for Ride {ride_id}.")


# --- Firestore Listeners ---

def on_pending_requests(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                match_driver(db_passenger, db_driver, change.document)


def on_proposal_updates(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            if not data:
                continue
            if data.get('status') == STATUS_ACCEPTED:
                print(f"[EVENT] Driver accepted {change.document.id}")
            elif data.get('status') in ['rejected', 'cancelled']:
                print(f"[EVENT] Driver rejected/cancelled {change.document.id}")


# --- Main Execution ---

if __name__ == '__main__':
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not db_passenger or not db_driver:
        print("FATAL: Firebase initialization failed.", file=sys.stderr)
        sys.exit(1)

    print("\n[SERVER STARTED] Listening for new ride requests and driver updates...")

    query_pending = db_passenger.collection('ride_requests').where('status', '==', STATUS_PENDING)
    query_pending.on_snapshot(on_pending_requests)

    query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
    query_proposals.on_snapshot(on_proposal_updates)

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nServer stopped.")
        sys.exit(0)
